In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
import os

print(tf.__version__)

In [ ]:
train_data = pd.read_csv('../data/ChatbotData_merged.csv')
train_data = train_data.dropna()
train_data.head()

In [ ]:
print('챗봇 샘플의 개수 :', len(train_data))
print(train_data.isnull().sum())

In [ ]:
questions = []
# 최대 길이를 학습 데이터 문장의 최대길이로 정의 - 43
MAX_LENGTH = 0
for sentence in train_data['Q']:
    # 구두점에 앞 띄어쓰기 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)
    length = len(sentence.split())
    if length > MAX_LENGTH:
      MAX_LENGTH = length

answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)
    length = len(sentence.split())
    if length > MAX_LENGTH:
      MAX_LENGTH = length

print(questions[:5])
print(answers[:5])
# print(MAX_LENGTH)

In [ ]:
# # 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
# tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
#     questions + answers, target_vocab_size=2**13)

In [ ]:
# corpus = []
# tokenizer_path = r'D:\banimo_diary\models\tokenizer.txt'
# with open(tokenizer_path, 'r', encoding='utf-8') as f:
#     for inx, line in enumerate(f):
#         if inx > 1:
#             corpus.append(line.strip())
# # 헤더 정보가 포함된 열 두개를 토크나이저 로드시에 제외하기 때문에 추가해줌
# corpus.insert(0, '')
# corpus.insert(0, '')
#
# new_tokenizer_path = tokenizer_path.replace('.subword', '_new.txt')
# with open(new_tokenizer_path, 'w', encoding='utf-8') as f:
#     f.write('\n'.join(corpus))
#     f.write('\n')
#

In [ ]:
corpus = []

tokenizer_path = r'D:\banimo_diary\models\vocab.txt'
with open(tokenizer_path, 'r', encoding='utf-8') as f:
    for line in f:
        corpus.append(line.strip())
print(corpus[:10])
tokenizer = tfds.deprecated.text.SubwordTextEncoder(vocab_list=corpus)

In [ ]:
tokenizer.save_to_file('checkpoints/vocab')

In [ ]:
# corpus = []
# with open('checkpoints/tokenizer.subwords', 'r', encoding='utf-8') as f:
#   for inx, line in enumerate(f):
#     if inx > 1:
#       # sent = sent.replace('\n', '')
#       # sent = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣?.!,_]+", " ", sent)
#       # sent = sent.strip()
#       corpus.append(line.strip())
#
# # corpus.insert(0, '### Metadata: {}')
# # corpus.insert(0, '### SubwordTextEncoder')
# # 헤더 정보가 포함된 열 두개를 토크나이저 로드시에 제외하기 때문에 추가해줌
# corpus.insert(0, '')
# corpus.insert(0, '')
# print(len(corpus))
# with open('checkpoints/tokenizer.txt', 'w', encoding='utf-8') as f:
#     f.write('\n'.join(corpus))
#     f.write('\n')

In [ ]:
print(tokenizer.vocab_size)

In [ ]:
# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

# print('시작 토큰 번호 :',START_TOKEN)
# print('종료 토큰 번호 :',END_TOKEN)
# print('단어 집합의 크기 :',VOCAB_SIZE)
#
# # 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
# # print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))

In [ ]:
# # 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# # 임의의 입력 문장을 sample_string에 저장
# sample_string = questions[20]
#
# # encode() : 텍스트 시퀀스 --> 정수 시퀀스
# tokenized_string = tokenizer.encode(sample_string)
# print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))
#
# # decode() : 정수 시퀀스 --> 텍스트 시퀀스
# original_string = tokenizer.decode(tokenized_string)
# print ('기존 문장: {}'.format(original_string))

In [ ]:
# # 각 정수는 각 단어와 어떻게 mapping되는지 병렬로 출력
# # 서브워드텍스트인코더는 의미있는 단위의 서브워드로 토크나이징한다. 띄어쓰기 단위 X 형태소 분석 단위 X
# for ts in tokenized_string:
#   print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

In [ ]:
# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)

print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

# 0번 샘플을 임의로 출력
print(questions[0])
print(answers[0])

In [ ]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 128
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# # 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
# print(answers[0]) # 기존 샘플
# print(answers[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 39가 된다.
# print(answers[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 39가 된다.

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

  def get_config(self):
    config = super().get_config()
    config.update({
        "pos_encoding": self.pos_encoding,
    })
    return config


In [ ]:
# @tf.keras.utils.register_keras_serializable()
# class PositionalEncoding(tf.keras.layers.Layer):
#     def __init__(self, position, d_model, **kwargs):
#         super(PositionalEncoding, self).__init__(**kwargs)
#         self.position = position
#         self.d_model = d_model
#         self.pos_encoding = self.positional_encoding(self.position, self.d_model)
#
#     def get_angles(self, position, i, d_model):
#         angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
#         return position * angles
#
#     def positional_encoding(self, position, d_model):
#         angle_rads = self.get_angles(
#             position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
#             i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
#             d_model=d_model)
#
#         # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
#         sines = tf.math.sin(angle_rads[:, 0::2])
#
#         # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
#         cosines = tf.math.cos(angle_rads[:, 1::2])
#
#         angle_rads = np.zeros(angle_rads.shape)
#         angle_rads[:, 0::2] = sines
#         angle_rads[:, 1::2] = cosines
#         pos_encoding = tf.constant(angle_rads)
#         pos_encoding = pos_encoding[tf.newaxis, ...]
#
#         print(pos_encoding.shape)
#         return tf.cast(pos_encoding, tf.float32)
#
#     def call(self, inputs):
#         return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]
#
#     def get_config(self):
#         return {
#             'position': self.position,
#             'd_model': self.d_model,
#         }
#
#     @classmethod
#     def from_config(cls, config):
#         return cls(**config)


In [ ]:
# # 문장의 길이 50, 임베딩 벡터의 차원 128
# sample_pos_encoding = PositionalEncoding(50, 128)
#
# plt.pcolormesh(sample_pos_encoding.pos_encoding.numpy()[0], cmap='RdBu')
# plt.xlabel('Depth')
# plt.xlim((0, 128))
# plt.ylabel('Position')
# plt.colorbar()
# plt.show()

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
    # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    # padding_mask : (batch_size, 1, 1, key의 문장 길이)

    # Q와 K의 곱. 어텐션 스코어 행렬.
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # 스케일링
    # dk의 루트값으로 나눠준다.
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
    # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
    if mask is not None:
        logits += (mask * -1e9)

    # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
    # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    output = tf.matmul(attention_weights, value)

    return output, attention_weights

In [ ]:
# # 임의의 Query, Key, Value인 Q, K, V 행렬 생성
# np.set_printoptions(suppress=True)
# temp_k = tf.constant([[10,0,0],
#                       [0,10,0],
#                       [0,0,10],
#                       [0,0,10]], dtype=tf.float32)  # (4, 3)
#
# temp_v = tf.constant([[   1,0],
#                       [  10,0],
#                       [ 100,5],
#                       [1000,6]], dtype=tf.float32)  # (4, 2)
# temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)

In [ ]:
# # 함수 실행
# temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
# print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
# print(temp_out) # 어텐션 값

In [ ]:
# temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)
# temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
# print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
# print(temp_out) # 어텐션 값

In [ ]:
# temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
# temp_out, temp_attn = scaled_dot_product_attention(temp_q, temp_k, temp_v, None)
# print(temp_attn) # 어텐션 분포(어텐션 가중치의 나열)
# print(temp_out) # 어텐션 값

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)
    return outputs

  def get_config(self):
    config = super().get_config()
    config.update({
        "num_heads": self.num_heads,
        "d_model": self.d_model,
        # "query_dense": self.query_dense,
        # "key_dense": self.key_dense,
        # "value_dense": self.value_dense,
    })
    return config

In [ ]:
# @tf.keras.utils.register_keras_serializable()
# class MyMultiHeadAttention(tf.keras.layers.Layer):
#
#     def __init__(self, d_model, num_heads, name="multi_head_attention", **kwargs):
#         super(MyMultiHeadAttention, self).__init__(name=name, **kwargs)
#         self.num_heads = num_heads
#         self.d_model = d_model
#
#         assert d_model % self.num_heads == 0
#
#         # d_model을 num_heads로 나눈 값.
#         # 논문 기준 : 64
#         self.depth = d_model // self.num_heads
#
#         # WQ, WK, WV에 해당하는 밀집층 정의
#         self.query_dense = tf.keras.layers.Dense(units=d_model)
#         self.key_dense = tf.keras.layers.Dense(units=d_model)
#         self.value_dense = tf.keras.layers.Dense(units=d_model)
#
#         # WO에 해당하는 밀집층 정의
#         self.dense = tf.keras.layers.Dense(units=d_model)
#
#   # num_heads 개수만큼 q, k, v를 split하는 함수
#     def split_heads(self, inputs, batch_size):
#         inputs = tf.reshape(
#             inputs, shape=(batch_size, -1, self.num_heads, self.depth))
#         return tf.transpose(inputs, perm=[0, 2, 1, 3])
#
#
#     def call(self, inputs):
#         query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
#         batch_size = tf.shape(query)[0]
#
#         # 1. WQ, WK, WV에 해당하는 밀집층 지나기
#         # q : (batch_size, query의 문장 길이, d_model)
#         # k : (batch_size, key의 문장 길이, d_model)
#         # v : (batch_size, value의 문장 길이, d_model)
#         # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
#         query = self.query_dense(query)
#         key = self.key_dense(key)
#         value = self.value_dense(value)
#
#         # 2. 헤드 나누기
#         # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
#         # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
#         # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
#         query = self.split_heads(query, batch_size)
#         key = self.split_heads(key, batch_size)
#         value = self.split_heads(value, batch_size)
#
#         # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
#         # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
#         scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
#         # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
#         scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
#
#         # 4. 헤드 연결(concatenate)하기
#         # (batch_size, query의 문장 길이, d_model)
#         concat_attention = tf.reshape(scaled_attention,
#                                       (batch_size, -1, self.d_model))
#
#         # 5. WO에 해당하는 밀집층 지나기
#         # (batch_size, query의 문장 길이, d_model)
#         outputs = self.dense(concat_attention)
#
#         return outputs
#
#
#     def get_config(self):
#         return {
#             'num_heads': self.num_heads,
#             'd_model': self.d_model,
#             'depth': self.depth,
#             'query_dense': self.query_dense,
#             'key_dense': self.key_dense,
#             'value_dense': self.value_dense,
#             'dense': self.dense
#         }
#
#     @classmethod
#     def from_config(cls, config):
#         return cls(**config)

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
# print(create_padding_mask(tf.constant([[1, 21, 777, 0, 0]])))

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
# print(create_look_ahead_mask(tf.constant([[1, 2, 0, 4, 5]])))

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def get_config(self):
        config = {
          'd_model': self.d_model,
          'warmup_steps': self.warmup_steps,
        }
        return config

    def __call__(self, step):
        step = tf.cast(step, tf.float32)  #####
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

from keras.callbacks import ModelCheckpoint

learning_rate = CustomSchedule(D_MODEL)
# learning_rate = customSchedule()

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
# def customSchedule():
#     initial_learning_rate = 0.1
#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#         initial_learning_rate,
#         decay_steps=100000,
#         decay_rate=0.96,
#         staircase=True)
#
#     return lr_schedule


In [ ]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)
  print(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  print('=====sentence=====', sentence)
  print('=====output=====', output)
  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    print('=====predictions=====\n', predictions)
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루       프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)
    print('=====output=====\n', output)
  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
'''No checkpoint'''
def train_model(epochs=50, mini_epoch=10):
    epoch_saved_path = './checkpoints/weights/epoch_saved.txt'
    if os.path.isfile(epoch_saved_path):
        with open(epoch_saved_path, 'r', encoding='utf-8') as f:
            total_epoch = int(f.read())
    else:
        total_epoch = 0

    # 학습된 최근 모델 불러오기
    model_path = f'checkpoints/weights/transformer_weight{total_epoch}.h5'
    sample_model_path = f'save/weights/model_sample.h5'
    if os.path.isfile(model_path):
        model.load_weights(model_path)
        print(f'{total_epoch}에폭 모델을 사용해 계속 학습합니다.')
    elif os.path.isfile(sample_model_path):
        model.load_weights(sample_model_path)
        print('저성능 샘플 모델을 사용해 학습합니다.')
    else:
        print('저장된 가중치가 없어 처음부터 학습합니다.')

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    EPOCHS = 100
    mini_epoch = 10

    for e in range(int(EPOCHS/mini_epoch)):
        model.fit(dataset, epochs=mini_epoch, workers=-1, callbacks=[early_stopping])
        total_epoch += mini_epoch
        with open(epoch_saved_path, 'w', encoding='utf-8') as f:
            f.write(str(total_epoch))
        print('>'*20, '샘플 예측', '<'*20)
        output = predict("오늘 엄마랑 에버랜드 갔다왔어.")
        print('>'*50)

        if total_epoch % 5 == 0:
            model_path = f'checkpoints/weights/transformer_weight{total_epoch}.h5'
            model.save_weights(model_path)

In [ ]:
train_model(epochs=30)

In [ ]:
# checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)
# # 모델 저장 문제 - unable to serialize to json -> eagerTensor type
# model.save(f'checkpoints/transformer_model_vanila_{EPOCHS}.h5')

# 모델 가중치 저장

# model.save_weights(f'checkpoints/transformer_weight_vanila{EPOCHS}')

In [ ]:
# model.save_weights(f'checkpoints/transformer_weight{EPOCHS}.h5')
# model.save_weights(f'checkpoints/transformer_weight_no_loss_{EPOCHS}')

In [ ]:
# # # 비상저장
# model.compile(optimizer=optimizer, loss=None)
# model.save_weights(f'checkpoints/transformer_weight_no_loss_complied_h5{EPOCHS}.h5')
# model.save_weights(f'checkpoints/transformer_weight_no_loss_compiled{EPOCHS}')

In [ ]:
# '''테스트중'''
# epoch_saved_path = './checkpoints/epoch_saved2.txt'
# if os.path.isfile(epoch_saved_path):
#     with open(epoch_saved_path, 'r', encoding='utf-8') as f:
#         epoch_saved = int(f.read())
# else:
#     epoch_saved = 0
#
# EPOCHS = 1
# filename = './checkpoints/checkpoint-epoch-{}-batch-{}.ckpt'.format(EPOCHS, BATCH_SIZE)
# checkpoint = ModelCheckpoint(filename,
#                              monitor='loss',
#                              verbose=1,
#                              # save_best_only=True,
#                              # save_weights_only=True,
#                              mode='auto',
#                             )
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
#
# if tf.train.latest_checkpoint('./checkpoints/'):
#     latest = tf.train.latest_checkpoint('./checkpoints/')
#     # 단어 임베딩 차원크기가 바뀌어서(vocab size를 늘리는 등) 저장된 파라미터와 shape안맞으면 에러발생
#     model.load_weights(latest)
#     print('존재하는 체크포인트로 학습진행 > ', latest)
# else:
#     print('저장된 체크포인트 없이 학습을 진행합니다')
#
# for e in range(EPOCHS):
#     model.fit(dataset, epochs=1, workers=-1, callbacks=[checkpoint, early_stopping])
#     epoch_saved += 1
#     with open('./checkpoints/epoch_saved2.txt', 'w', encoding='utf-8') as f:
#         f.write(str(epoch_saved))
#     print('>'*20, '샘플 예측', '<'*20)
#     output = predict("오늘 엄마랑 롯데월드 갔다왔어.")
#     print('>'*50)
#
#
# model.compile(optimizer=optimizer, loss=None)
# model.save('checkpoints/transformer_model__babo.h5')
# # 모델 가중치 저장
# model.save_weights('checkpoints/transformer_weight__babo')

In [ ]:
# epoch_saved_path = './checkpoints/epoch_saved.txt'
# if os.path.isfile(epoch_saved_path):
#     with open(epoch_saved_path, 'r', encoding='utf-8') as f:
#         epoch_saved = int(f.read())
# else:
#     epoch_saved = 0
#
# EPOCHS = 100
# filename = './checkpoints/checkpoint-epoch-{}-batch-{}.ckpt'.format(EPOCHS, BATCH_SIZE)
# checkpoint = ModelCheckpoint(filename,
#                              monitor='loss',
#                              verbose=1,
#                              save_best_only=True,
#                              save_weights_only=True,
#                              mode='auto',
#                             )
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
#
# if tf.train.latest_checkpoint('./checkpoints/'):
#     latest = tf.train.latest_checkpoint('./checkpoints/')
#     # 단어 임베딩 차원크기가 바뀌어서(vocab size를 늘리는 등) 저장된 파라미터와 shape안맞으면 에러발생
#     model.load_weights(latest)
#     print('존재하는 체크포인트로 학습진행 > ', latest)
# else:
#     print('저장된 체크포인트 없이 학습을 진행합니다')
#
# for e in range(0, int(EPOCHS/5)):
#     model.fit(dataset, epochs=5, workers=-1, callbacks=[checkpoint, early_stopping])
#     epoch_saved += 5
#     with open('./checkpoints/epoch_saved2.txt', 'w', encoding='utf-8') as f:
#         f.write(str(epoch_saved))
#     print('>'*20, '샘플 예측', '<'*20)
#     output = predict("오늘 엄마랑 롯데월드 갔다왔어.")
#     print('>'*50)

In [ ]:
model.summary()

In [ ]:
# print(total_epoch)
output = predict("비가 와서 날씨가 흐린데 그래도 나는 기분이 좋아.")
# output = predict("아 화난다.")
# output = predict("나 엄마보러 제주도에 가고싶어.")
# output = predict("너 맞을래?")

In [1]:
model.load_weights(r'D:\banimo_diary\models\save\weights\transformer_weight100.h5')
output = predict('아주 지긋지긋해!')

NameError: name 'model' is not defined